In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для установления степени взаимосвязи между рассматриваемыми показателями *необходимо*:
1. Перевести количественную переменную 'children' в категориальную:
    - Разделить на несколько категорий: 'без детей', 'до 3-ех детей', 'более 3-ех детей'.
    Для этого создадим отдельный столбец в датафрейме, который будет включать в себя данные категории.
2. Создать сводную таблицу для возможности сравнения двух показателей

In [22]:
def child_def(children):
    if 1<=children<=2:
        return 'до 2-х детей'
    if children >=3:
        return 'более 3-ех детей'
    return 'без детей' #функция категоризации

In [23]:
data['child_categorize'] = data['children'].apply(child_def) #добавляем новый столбец

In [24]:
data.head() #проверяем наличие нового столбца, выведем первые 5 значений

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,child_categorize
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,до 2-х детей
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,до 2-х детей
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,более 3-ех детей
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,без детей


In [25]:
child_debt = data.pivot_table(index='child_categorize', columns='debt', values='gender', aggfunc='count')
#создаем сводную таблицу, где по столбцу 'children_rank' 
#и по значению столбца 'debt' будет происходить группировка данных
child_debt.columns = ['no_debt', 'debt'] #создаем столбцы 
child_debt['share'] = child_debt['debt'] * 100 / (child_debt['debt'] + child_debt['no_debt']) #находим долю должников
child_debt.sort_values(by='share')#сортируем по столбцу 'share' по возрастанию

,no_debt,debt,share
child_categorize,,,
без детей,13028,1063,7.543822
более 3-ех детей,349,31,8.157895
до 2-х детей,6222,638,9.300292


**Вывод:** *Доля должников, у которых не более 2-х детей, имеет наибольший удельный вес.* 
Можно предположить, что далеко не каждая семья с 1-2 детьми способна справиться с той финансовой нагрузкой, которая возникает в связи с появлением дополнительных расходов на содержание детей. Вместе с этим, доля должников среди многодетных семей, где более 3-ех детей (8.15%), ниже того же показателя среди должников в категории "до 2-х детей"(9.3%) на 1,15%. Возможно, это связано с более рациональным планированием доходов и расходов в данных семьях, т.к наличие 3-го ребенка можно объяснить желанием, а главное **возможностью родителей** обеспечить всех своих детей всем необходимым; также данную тенденцию можно объяснить существованием государственной поддержки многодетных семей, что в свою очередь способно снизить финансовую нагрузку. Показатель семей без детей (7.54%) имеет наименьший удельный вес, что логически объясняется отсутствием финансовой нагрузки на содержание детей.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
data['family_status'].unique() #выведем все уникальные значение столбца

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [27]:
family_debt = data.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
#создаем сводную таблицу, где по столбцу 'family_status' 
#и по значению столбца 'debt' будет происходить группировка данных
family_debt.columns = ['no_debt', 'debt'] #создаем столбцы 
family_debt['share'] = family_debt['debt'] * 100 / (family_debt['debt'] + family_debt['no_debt']) #находим долю должников
family_debt.sort_values(by='share')#сортируем по столбцу 'share' по возрастанию

,no_debt,debt,share
family_status,,,
вдовец / вдова,888,63,6.624606
в разводе,1105,84,7.064760
женат / замужем,11334,927,7.560558
гражданский брак,3749,385,9.313014
Не женат / не замужем,2523,273,9.763948


**Вывод:** *Доля должников в статусе "Не женат / не замужем" имеет наибольший удельный вес.* 
Стоит также отметить, что доля должников, проживающих в гражданском браке (9.31%), имеет одно из наибольших значений и почти равно  наибольшему показателю (9.76%). Это можно объяснить более фривольным планированием доходов и расходов, не обремененных институтом семьи. Так, доля должников, которые находятся (7.56%) или пребывали в браке (7.06%), гораздо ниже предыдущих показателей. Среди вдов и вдовцов показатель имеет наименьший результат (6.62%), что объясняется более рациональным подходом к планированию бюджета, а также наличием большого опыта расходования средств во всех предыдущих семейных статусах.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [28]:
income_debt = data.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')
#создаем сводную таблицу, где по столбцу 'total_income_category' 
#и по значению столбца 'debt' будет происходить группировка данных
income_debt.columns = ['no_debt', 'debt'] #создаем столбцы 
income_debt['share'] = income_debt['debt'] * 100 / (income_debt['debt'] + income_debt['no_debt']) #находим долю должников
income_debt.sort_values(by='share')#сортируем по столбцу 'share' по возрастанию

,no_debt,debt,share
total_income_category,,,
D,328,21,6.017192
B,4660,354,7.060231
A,23,2,8.000000
C,14568,1353,8.498210
E,20,2,9.090909


**Вывод:** *Доля должников среди людей с уровнем дохода от 50001-200000 **категория С** имеет наибольший удельный вес.*
Важно отметить, что категории А и Е не являются репрезентативными, т.к. выборка не достаточно высокая для наиболее объективного анализа. Следовательно, остается рассматривать категорию С (50001-200000)(8.49%), которая является наибольшей, а также В (доход: 200001–1000000)(7.06%) и D (30001–50000)(6.01%). Выводы можно сделать следующие: в категорию С входит средний уровень дохода, а также наибольшее количество участников, поэтому доля должников наибольшая; категория В с самим высоким доходом, при этом доля должников значительно ниже относительно категории С, в категории D (с самым низким доходом) наименьшее число должников, что можно объяснить высоким уровнем отказов от банка в выдаче данной категории кредитов. 




#### 3.4 Как разные цели кредита влияют на его возврат в срок?

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [29]:
purpose_debt = data.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')
#создаем сводную таблицу, где по столбцу 'purpose_category' 
#и по значению столбца 'debt' будет происходить группировка данных
purpose_debt.columns = ['no_debt', 'debt'] #создаем столбцы 
purpose_debt['share'] = purpose_debt['debt'] * 100 / (purpose_debt['debt'] + purpose_debt['no_debt']) #находим долю должников 
purpose_debt.sort_values(by='share')#сортируем по столбцу 'share' по возрастанию

,no_debt,debt,share
purpose_category,,,
операции с недвижимостью,9971,780,7.255139
проведение свадьбы,2130,183,7.911803
получение образования,3619,369,9.252758
операции с автомобилем,3879,400,9.347978


**Вывод:** *Доли должников среди автомобилистов и желающих получить образовние имеют наибольший удельный вес.*
Вместе с этим доли среди желающих провести свадьбу и операции с недвижимостью относительно ниже. Последняя цель на получение кредита заставляет более ответственно подходить к вопросу выплаты кредита и планирования своего бюджета.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Одни из причин: технические проблемы: повреждение файла при сохранении, сбой работы программы и др., а также человеческий фактор: невнимательность и т.д.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* медиана представляет собой число в выборке, между двумя половинами: одна больше
него, а другая — меньше. Следовательно, она идеально подходит для замены аномальных значений.

# Общий вывод.

#### Предобработка данных
*В рамках предобработки данных были произведены следующие операции*: 
- удаление пропусков и их заполнение медианным значением по столбцу;
- обработка аномальных значений;
- изменение типов данных;
- обработка дупликатов;
- категоризация данных.

#### Ответы на поставленные вопросы в рамках исследования данных
1. **Была выявлена зависимость между количеством детей и невозвратом кредита в срок.**  
Так, наличие одного или двух детей повышает вероятность непогашения долга.

2. **Была выявлена зависимость между семейным положением и невозвратом кредита в срок.**  
Должников среди людей, которые не состоят/не состояли в браке или брак которых не зарегистрирован официально, наибольшее число.

3. **Была выявлена зависимость между уровнем дохода и невозвратом кредита в срок.**  
Банк выдает наибольшее число кредитов людям со средним заработком, что и объясняется наибольшей долей должников среди данной категории.

4. **Существует взаимосвязь между целями на получение кредита и сроком его возврата**: люди, которые берут кредиты на операции с авто или получение образования, имеют большие шансы стать должниками.

5. **Причины появления пропусков в исходных данных можно разделить на две категории**: технические и человеческий фактор.

6. **Медиана наиболее устойчива к аномальным изменениям в общем массиве данных.**